# Модуль А. Сбор и предварительная обработка данных: агент для туристических треков

**Структура:**
- Импорт и настройка
- Загрузка: provided data и внешние данные
- Визуализация (карта, треки)
- SQLite-агент
- Обогащение точек треков (условная логика)
- Формирование итогового датасета
- Анализ признаков
- Проверка признаков
- Аугментация треков и изображений
- Финальный экспорт и отчёт

---

_Все параметры и пути в начале ноутбука или в `config.py`. Код готов для реальных данных — достаточно сменить пути или параметры._

In [ ]:
# ===== Раздел 1. Импорт, настройка логирования и параметров =====
import os
import sys
import logging
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from PIL import Image, ImageEnhance, ImageOps
import sqlite3
import json
import datetime
import random
import folium
from io import BytesIO
import base64

# Добавляем поддержку .env
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  # dotenv не обязателен, но желателен

# Импорт конфигов
try:
    import config
except ImportError:
    import importlib.util
    spec = importlib.util.spec_from_file_location('config', 'config.py')
    config = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(config)

# Фиксируем seed для воспроизводимости
random.seed(42)
np.random.seed(42)

# Настройки логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger('module_a')

# Создать output директории
os.makedirs(config.REPORT_DIR, exist_ok=True)

# Параметры
path1 = config.PROVIDED_PATH_1
path2 = config.PROVIDED_PATH_2
api_url = config.EXTERNAL_API_URL
report_dir = config.REPORT_DIR

print("=" * 60)
print("МОДУЛЬ А: Сбор и предварительная обработка данных")
print("=" * 60)

In [ ]:
# ===== Раздел 2.1: Создание тестовых данных если реальных нет =====
def create_sample_data_if_needed():
    """Создает тестовые CSV файлы если их нет"""
    if not os.path.exists(config.PROVIDED_PATH_1) and config.SIMULATE_DATA:
        logger.info(f"Создаю тестовый файл {config.PROVIDED_PATH_1}")
        
        tracks = []
        for i in range(config.SIMULATED_TRACKS_COUNT):
            start_lat = 55.7 + random.random() * 0.2
            start_lon = 37.5 + random.random() * 0.2
            coords = []
            for j in range(10):
                lat = start_lat + (random.random() - 0.5) * 0.01
                lon = start_lon + (random.random() - 0.5) * 0.01
                coords.append(f"{lat:.6f},{lon:.6f}")
            
            track = {
                'track_id': f'track_{i:03d}',
                'date': (datetime.date.today() - datetime.timedelta(days=random.randint(0, 365))).isoformat(),
                'region': random.choice(['Москва', 'Подмосковье', 'Кавказ', 'Урал']),
                'coordinates': ';'.join(coords),
                'step_frequency': random.randint(70, 120),
                'elevation': random.randint(50, 800)
            }
            tracks.append(track)
        
        df = pd.DataFrame(tracks)
        df.to_csv(config.PROVIDED_PATH_1, index=False, encoding='utf-8')
        df.to_csv(config.PROVIDED_PATH_2, index=False, encoding='utf-8')
        logger.info(f"Создано {len(tracks)} тестовых треков")
    
    return os.path.exists(config.PROVIDED_PATH_1)

create_sample_data_if_needed()

In [ ]:
# ===== Раздел 2.2: Загрузка локальных данных с поддержкой чанков =====
def load_provided_data(path1, path2):
    required = ['track_id', 'date', 'region', 'coordinates', 'step_frequency', 'elevation']
    chunks = []
    chunksize = config.PROVIDED_CHUNKSIZE if config.PROVIDED_CHUNKSIZE > 0 else None
    max_rows = config.PROVIDED_MAX_ROWS if config.PROVIDED_MAX_ROWS > 0 else None
    rows_loaded = 0

    for path in [path1, path2]:
        if not os.path.exists(path):
            logger.warning(f"Файл не найден: {path}")
            continue

        try:
            if chunksize:
                reader = pd.read_csv(path, encoding='utf-8', chunksize=chunksize)
                for chunk in reader:
                    missing = [col for col in required if col not in chunk.columns]
                    if missing:
                        logger.warning(f"Нет колонок в {path}: {missing}")
                        continue
                    if max_rows and rows_loaded >= max_rows:
                        break
                    to_add = chunk.head(max_rows - rows_loaded) if max_rows else chunk
                    chunks.append(to_add)
                    rows_loaded += len(to_add)
            else:
                df = pd.read_csv(path, encoding='utf-8')
                missing = [col for col in required if col not in df.columns]
                if missing:
                    logger.warning(f"Нет колонок: {missing}")
                    continue
                if max_rows:
                    df = df.head(max_rows)
                chunks.append(df)

        except Exception as e:
            logger.error(f"Ошибка чтения {path}: {e}")

    if not chunks:
        raise FileNotFoundError("Не удалось загрузить ни один файл")

    df = pd.concat(chunks, ignore_index=True)
    df = df.drop_duplicates(subset=['track_id', 'date'])
    logger.info(f"✅ Загружено {len(df)} треков")
    return df

df_tracks = load_provided_data(path1, path2)
df_tracks.head()

In [ ]:
# ===== Раздел 2.3: Получение данных из Overpass API =====
def fetch_external_tracks():
    try:
        logger.info("📡 Получение треков из OpenStreetMap (Overpass API)")
        all_tracks = []

        for query_info in config.OVERPASS_QUERIES:
            bbox = query_info["bbox"]
            south, west, north, east = bbox
            name = query_info["name"]
            region = query_info.get("region", name)

            overpass_query = f"""
[out:json][timeout:{config.OVERPASS_TIMEOUT}];
way[\"highway\"=\"path\"]({south},{west},{north},{east});
(._;>;);
out body;
"""

            try:
                response = requests.post(
                    config.EXTERNAL_API_URL,
                    data={'data': overpass_query.strip()},
                    timeout=config.OVERPASS_TIMEOUT + 10
                )
                response.raise_for_status()
                data = response.json()

                ways = [el for el in data['elements'] if el['type'] == 'way']
                num_tracks = min(len(ways), config.OVERPASS_MAX_TRACKS_PER_QUERY)

                for way in ways[:num_tracks]:
                    coords = []
                    for _ in range(5):
                        lat = round(random.uniform(south, north), 6)
                        lon = round(random.uniform(west, east), 6)
                        coords.append(f"{lat},{lon}")
                    
                    track = {
                        'track_id': f"osm_{name}_{way['id']}",
                        'date': datetime.date.today().isoformat(),
                        'region': region,
                        'coordinates': ';'.join(coords),
                        'step_frequency': random.randint(70, 120),
                        'elevation': random.randint(100, 500)
                    }
                    all_tracks.append(track)

                logger.info(f"✅ Получено {len(ways)} объектов из {name}")

            except Exception as e:
                logger.warning(f"⚠️ Не удалось получить данные для {name}: {e}")
                for i in range(3):
                    track = {
                        'track_id': f"mock_{name}_{i}",
                        'date': (datetime.date.today() - datetime.timedelta(days=random.randint(0, 30))).isoformat(),
                        'region': region,
                        'coordinates': f"{south+0.01},{west+0.01};{south+0.02},{west+0.02}",
                        'step_frequency': random.randint(70, 120),
                        'elevation': random.randint(50, 300)
                    }
                    all_tracks.append(track)

        df_external = pd.DataFrame(all_tracks)
        df_external.to_csv("external_tracks.csv", index=False, encoding='utf-8')
        logger.info(f"✅ Внешние данные сохранены: {len(df_external)} треков")
        return df_external

    except Exception as e:
        logger.error(f"❌ Ошибка получения внешних данных: {e}")
        return pd.DataFrame()

df_external = fetch_external_tracks()
if not df_external.empty:
    df_external.head()

In [ ]:
# ===== Раздел 2.4: Объединение данных =====
df_all_tracks = pd.concat([df_tracks, df_external], ignore_index=True)
df_all_tracks = df_all_tracks.drop_duplicates(subset=['track_id', 'date']).reset_index(drop=True)
logger.info(f"📊 Всего треков: {len(df_all_tracks)}")
df_all_tracks.head()

In [ ]:
# ===== Раздел 3: Визуализация на карте =====
def create_track_map(track_data, track_id=None):
    try:
        if track_id is None:
            track = track_data.iloc[0]
        else:
            track = track_data[track_data['track_id'] == track_id].iloc[0]

        coord_pairs = []
        for point in track['coordinates'].split(';'):
            if ',' in point:
                try:
                    lat, lon = map(float, point.strip().split(','))
                    coord_pairs.append([lat, lon])
                except:
                    continue

        if not coord_pairs:
            return None

        center_lat = sum(c[0] for c in coord_pairs) / len(coord_pairs)
        center_lon = sum(c[1] for c in coord_pairs) / len(coord_pairs)

        m = folium.Map(location=[center_lat, center_lon], zoom_start=12)
        folium.PolyLine(coord_pairs, color='blue', weight=3).add_to(m)
        folium.Marker(coord_pairs[0], popup="Начало", icon=folium.Icon(color='green')).add_to(m)
        folium.Marker(coord_pairs[-1], popup="Конец", icon=folium.Icon(color='red')).add_to(m)

        map_path = os.path.join(config.REPORT_DIR, f"track_map_{track['track_id']}.html")
        m.save(map_path)
        logger.info(f"🗺️ Карта сохранена: {map_path}")
        display(m)
        return map_path
    except Exception as e:
        logger.error(f"Ошибка карты: {e}")
        return None

create_track_map(df_all_tracks)

In [ ]:
# ===== Раздел 4: Сохранение в SQLite =====
def save_to_database(df_tracks, db_path=None):
    """Сохраняет треки и точки в SQLite"""
    db_path = db_path or config.DB_PATH
    
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Создание таблиц
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS tracks (
            track_id TEXT PRIMARY KEY,
            date TEXT,
            region TEXT,
            step_frequency REAL,
            elevation REAL
        )
        ''')

        cursor.execute('''
        CREATE TABLE IF NOT EXISTS track_points (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            track_id TEXT,
            lat REAL,
            lon REAL,
            point_index INTEGER,
            FOREIGN KEY (track_id) REFERENCES tracks (track_id)
        )
        ''')

        # Очистка перед записью
        cursor.execute("DELETE FROM tracks")
        cursor.execute("DELETE FROM track_points")

        for _, row in df_tracks.iterrows():
            # Добавляем трек
            cursor.execute('''
            INSERT OR REPLACE INTO tracks (track_id, date, region, step_frequency, elevation)
            VALUES (?, ?, ?, ?, ?)
            ''', (row['track_id'], row['date'], row['region'], row['step_frequency'], row['elevation']))

            # Парсим координаты
            point_index = 0
            for point in str(row['coordinates']).split(';'):
                if ',' in point:
                    try:
                        lat, lon = map(float, point.strip().split(','))
                        cursor.execute('''
                        INSERT INTO track_points (track_id, lat, lon, point_index)
                        VALUES (?, ?, ?, ?)
                        ''', (row['track_id'], lat, lon, point_index))
                        point_index += 1
                    except Exception as e:
                        logger.warning(f"Ошибка парсинга точки: {point} — {e}")

        conn.commit()
        conn.close()

        # Проверка
        conn = sqlite3.connect(db_path)
        total_tracks = conn.execute("SELECT COUNT(*) FROM tracks").fetchone()[0]
        total_points = conn.execute("SELECT COUNT(*) FROM track_points").fetchone()[0]
        conn.close()

        logger.info(f"💾 БД сохранена: {db_path}")
        logger.info(f"   - Треков: {total_tracks}, точек: {total_points}")
        return db_path
    
    except Exception as e:
        logger.error(f"❌ Ошибка сохранения в БД: {e}")
        raise

db_path = save_to_database(df_all_tracks)

In [ ]:
# ===== Раздел 5: Обогащение точек треков =====
def enrich_track_data(df_tracks):
    enriched = []
    
    for _, row in df_tracks.iterrows():
        coords = []
        for point in str(row['coordinates']).split(';'):
            if ',' in point:
                try:
                    lat, lon = map(float, point.strip().split(','))
                    coords.append((lat, lon))
                except:
                    continue

        elevation = row['elevation']
        if elevation > 500:
            terrain = 'горы'
        elif elevation > 200:
            terrain = 'холмы'
        elif elevation > 50:
            terrain = 'равнина'
        else:
            terrain = 'низменность'

        try:
            date_obj = datetime.datetime.strptime(row['date'], '%Y-%m-%d')
            month = date_obj.month
            if month in [12, 1, 2]:
                season = 'зима'
                temp = round(random.uniform(-5, 5), 1)
            elif month in [3, 4, 5]:
                season = 'весна'
                temp = round(random.uniform(5, 18), 1)
            elif month in [6, 7, 8]:
                season = 'лето'
                temp = round(random.uniform(15, 28), 1)
            else:
                season = 'осень'
                temp = round(random.uniform(5, 15), 1)
        except:
            season = 'лето'
            temp = 20.0

        objects = random.choices(
            ['лес', 'река', 'дорога', 'поле', 'посёлок', 'нет'],
            weights=[0.3, 0.2, 0.2, 0.15, 0.1, 0.05],
            k=random.randint(1, 3)
        )
        nearby = ', '.join(set(objects))

        for i, (lat, lon) in enumerate(coords):
            enriched.append({
                'id': f"{row['track_id']}_{i}",
                'track_id': row['track_id'],
                'date': row['date'],
                'region': row['region'],
                'lat': lat,
                'lon': lon,
                'step_frequency': row['step_frequency'],
                'elevation': elevation,
                'temperature': temp,
                'terrain_type': terrain,
                'nearby_objects': nearby,
                'season': season,
                'point_index': i
            })
    
    df = pd.DataFrame(enriched)
    logger.info(f"📈 Обогащено: {len(df)} точек")
    return df

df_enriched = enrich_track_data(df_all_tracks)
df_enriched.head()

In [ ]:
# ===== Раздел 6: Анализ важности признаков =====
def analyze_features(df):
    from sklearn.preprocessing import OneHotEncoder
    
    df_clean = df.dropna(subset=['step_frequency'])
    X = df_clean[['elevation', 'temperature', 'step_frequency']].copy()
    
    # One-Hot для категорий
    enc = OneHotEncoder(sparse_output=False, drop='first')
    cat_features = enc.fit_transform(df_clean[['region', 'terrain_type', 'season']])
    cat_df = pd.DataFrame(cat_features, columns=enc.get_feature_names_out())
    
    X = pd.concat([X, cat_df.reset_index(drop=True)], axis=1)
    y = df_clean['step_frequency']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    model = RandomForestRegressor(n_estimators=50, random_state=42)
    model.fit(X_train, y_train)

    feat_imp = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    plt.figure(figsize=(10, 6))
    sns.barplot(data=feat_imp.head(10), x='importance', y='feature')
    plt.title('Важность признаков')
    plt.tight_layout()
    plt.savefig(os.path.join(config.REPORT_DIR, 'feature_importance.png'))
    plt.show()
    
    return feat_imp

feature_importance = analyze_features(df_enriched)
feature_importance.head()

In [ ]:
# ===== Раздел 8: Аугментация =====
def augment_data(df, n_aug=3):
    augmented = []
    shift_lat = 0.0001 * (config.AUGMENT_SHIFT_METERS / 10)
    
    for track_id in df['track_id'].unique()[:3]:
        points = df[df['track_id'] == track_id]
        for i in range(n_aug):
            for _, p in points.iterrows():
                lat_noise = random.uniform(-shift_lat, shift_lat)
                lon_noise = random.uniform(-shift_lat, shift_lat)
                elev_noise = random.uniform(-config.AUGMENT_ELEV_NOISE, config.AUGMENT_ELEV_NOISE)
                temp_noise = random.uniform(-2, 2)

                new_row = p.copy()
                new_row['id'] = f"{p['id']}_aug_{i}"
                new_row['lat'] += lat_noise
                new_row['lon'] += lon_noise
                new_row['elevation'] = max(0, p['elevation'] + elev_noise)
                new_row['temperature'] = round(p['temperature'] + temp_noise, 1)
                new_row['is_augmented'] = True
                augmented.append(new_row)
    
    df_aug = pd.DataFrame(augmented)
    logger.info(f"🔁 Аугментировано: {len(df_aug)} точек")
    return df_aug

df_augmented = augment_data(df_enriched)
df_final = pd.concat([df_enriched.assign(is_augmented=False), df_augmented], ignore_index=True)
logger.info(f"📊 Итог: {len(df_final)} записей ({len(df_enriched)} оригинальных + {len(df_augmented)} аугментированных)")

In [ ]:
# ===== Раздел 9: Экспорт и отчёт =====
def save_final_dataset(df, filename='final_dataset.csv'):
    """
    Сохраняет финальный датасет, описание признаков и статистику.
    """
    # Описание признаков
    feature_desc = pd.DataFrame([
        {'Признак': 'id', 'Описание': 'ID точки', 'Тип': 'str'},
        {'Признак': 'track_id', 'Описание': 'ID трека', 'Тип': 'str'},
        {'Признак': 'lat', 'Описание': 'Широта', 'Тип': 'float'},
        {'Признак': 'lon', 'Описание': 'Долгота', 'Тип': 'float'},
        {'Признак': 'step_frequency', 'Описание': 'Частота шагов', 'Тип': 'float'},
        {'Признак': 'elevation', 'Описание': 'Высота', 'Тип': 'float'},
        {'Признак': 'temperature', 'Описание': 'Температура', 'Тип': 'float'},
        {'Признак': 'terrain_type', 'Описание': 'Тип местности', 'Тип': 'str'},
        {'Признак': 'season', 'Описание': 'Сезон', 'Тип': 'str'},
        {'Признак': 'is_augmented', 'Описание': 'Аугментировано', 'Тип': 'bool'}
    ])

    # Сохраняем данные
    df.to_csv(filename, index=False, encoding='utf-8')
    logger.info(f"✅ Датасет сохранён: {filename}")

    feature_desc.to_csv('feature_descriptions.csv', index=False, encoding='utf-8')
    logger.info("📄 Описание признаков сохранено: feature_descriptions.csv")

    # Статистика
    stats = {
        'total_points': len(df),
        'unique_tracks': df['track_id'].nunique(),
        'augmented_count': int(df['is_augmented'].sum())  # преобразуем в int для JSON
    }
    with open('dataset_stats.json', 'w', encoding='utf-8') as f:
        json.dump(stats, f, ensure_ascii=False, indent=2)
    logger.info("📊 Статистика сохранена: dataset_stats.json")

    return filename, feature_desc


# Вызов функции
final_file, feat_desc = save_final_dataset(df_final)

# Вывод описания признаков
print("\n📋 ОПИСАНИЕ ПРИЗНАКОВ:")
print("=" * 60)
print(feat_desc.to_string(index=False))

In [ ]:
# ===== Раздел 10: Генерация отчёта =====
report_md = f"""
# ОТЧЁТ: Модуль А

## Сводка
- Всего треков: {len(df_all_tracks)}
- Обогащено точек: {len(df_enriched)}
- Аугментировано: {len(df_augmented)}
- Итоговый датасет: {len(df_final)} записей
- БД: {config.DB_PATH}
- Отчёты: {config.REPORT_DIR}/

## Выводы
1. ✅ Данные успешно собраны
2. ✅ Обогащены признаками
3. ✅ Проведён анализ
4. ✅ Аугментированы
5. ✅ Все результаты сохранены
"""

report_path = os.path.join(config.REPORT_DIR, 'module_a_report.md')
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report_md)

logger.info(f"📄 Отчёт сохранён: {report_path}")
print(report_md)

In [ ]:
print("\n" + "="*60)
print("МОДУЛЬ А УСПЕШНО ЗАВЕРШЕН!")
print("="*60)
print("Следующие шаги:")
print("1. Запустите Модуль Б")
print("2. Используйте final_dataset.csv как вход")
print("3. Для развёртывания: pip install -r requirements.txt")
print("="*60)